# Graph Exploration: Applications

## Graphs

We use again `networkx` to represent and draw graphs:

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

edges = [(0,1), (0,3), (1,2), (1,3), (2,4), 
         (2,5), (3,2), (3,4), (4,1), (4,5)]

directed_graph = nx.DiGraph()
directed_graph.add_nodes_from([0,1,2,3,4,5])
directed_graph.add_edges_from(edges)

undirected_graph = nx.Graph()
undirected_graph.add_nodes_from([0,1,2,3,4,5])
undirected_graph.add_edges_from(edges)

# g = nx.gnm_random_graph(8, 16, directed=True)

In [ ]:
pos = nx.spring_layout(directed_graph)
nx.draw(directed_graph, pos, with_labels=True, 
        node_size=1000, node_color='lightblue')

## Depth-first and Breadth-first Search

Since we need them for the different applications, we include again the implementation of DFS and BFS from the last lecture (since we will later need it, DFS stores the reverse postorder of the vertices).

In [ ]:
from collections import deque

def depth_first_exploration(graph, node, visited=None, 
                            reverse_postorder=None):
    if visited is None:
        visited = set()
    if node in visited:
        return
    visited.add(node)
    
    for s in graph.successors(node):
        depth_first_exploration(graph, s, visited, 
                                reverse_postorder)
    if reverse_postorder is not None:
        reverse_postorder.appendleft(node)

order = deque()
depth_first_exploration(directed_graph, 0, reverse_postorder=order)
print(order)

In [ ]:
def bfs_with_predecessors(graph, node):
    predecessor = dict()
    queue = deque()
    # use self-loop for start node
    predecessor[node] = node
    queue.append(node)
    while queue:
        v = queue.popleft()
    
        for s in graph.successors(v):
            if s not in predecessor:
                predecessor[s] = v
                queue.append(s)
    return predecessor

## Shortest Paths

For unweighted graphs, we can determine shortest paths by means of a breadth-first search, storing the predecessors in the induced search tree. We extract the path backwards from the target to the start vertex.

Since we also want to draw the path, we do not extract the sequence of vertices but the sequence of edges.

In [ ]:
def shortest_path(graph, from_node, to_node):
    # more efficient: stop bfs as soon as to_node has been visited
    predecessor = bfs_with_predecessors(directed_graph, from_node)
    if to_node not in predecessor:
        print("There is no path from", from_node, "to", to_node)
        return None
    
    path = deque()
    current_node = to_node
    pre = predecessor[current_node]
    while pre is not current_node:
        # pre is current_node only for from_node
        path.appendleft((pre, current_node))
        current_node = pre
        pre = predecessor[current_node]
    return path

In [ ]:
path = shortest_path(directed_graph, 0, 5)
print(path)
if path is not None:
    nx.draw(directed_graph, pos, with_labels=True, 
            node_size=1000, node_color='lightblue')
    nx.draw_networkx_edges(directed_graph, pos, 
                           edgelist=path, width=8, alpha=0.5, 
                           edge_color="red")


## Cycles

A directed acyclic graph (DAG) is a directed graph that does not contain a directed cycle. For instance:

In [ ]:
dag = nx.DiGraph()
dag.add_nodes_from([0,1,2,3,4,5,6])
dag.add_edges_from([(0,5),(1,0),(1,5),(2,5),(3,2),(4,2),
                    (4,3),(6,1),(6,3)])

pos_dag = nx.spring_layout(dag)
nx.draw(dag, pos_dag, with_labels=True, node_size=1000, 
        node_color='lightblue')

The cycle detection consists of a number of breadth-first explorations, that test the induced search tree for back edges.

In [ ]:
class DirectedCycle:
    def __init__(self, graph):
        self.predecessor = dict()
        self.on_current_path = set()
        self.cycle = None
        for node in graph.nodes:
            if self.has_cycle():
                break
            if node not in self.predecessor:
                self.predecessor[node] = node
                self.dfs(graph, node)

    def has_cycle(self):
        return self.cycle is not None

    def dfs(self, graph, node):
        self.on_current_path.add(node)
        for s in graph.successors(node):
            if self.has_cycle():
                return
            if s in self.on_current_path:
                self.predecessor[s] = node
                self.extract_cycle(s)
            if s not in self.predecessor:
                self.predecessor[s] = node
                self.dfs(graph, s)
        self.on_current_path.remove(node)
        
    def extract_cycle(self, node):
        self.cycle = deque()
        current = node
        self.cycle.appendleft(current)
        while True:
            current = self.predecessor[current]
            self.cycle.appendleft(current)
            if current == node:
                return    

In [ ]:
cycle_detector = DirectedCycle(dag)
print("Does the DAG contain a cycle?", cycle_detector.has_cycle())
cycle_detector = DirectedCycle(directed_graph)
print("Does the other graph contain a cycle?", cycle_detector.has_cycle())
cycle = cycle_detector.cycle
print(cycle)

path = [(cycle[i], cycle[i+1]) for i in range(len(cycle)-1)]
nx.draw(directed_graph, pos, with_labels=True, node_size=1000, node_color='lightblue')
_ = nx.draw_networkx_edges(directed_graph, pos, edgelist=path, width=8, alpha=0.5, edge_color="red")


## Topological Sort

A topological sort of a directed, acyclic graph is a linear ordering of all its vertices such that if there is an edge from u to v, then u appears before v in the ordering.

To compute a topological sort, we use a sequence of DFS explorations and extract a *complete* reverse postorder.

In [ ]:
def topological_sort(acyclic_digraph):
    visited = set()
    reverse_postorder = deque()
    for node in acyclic_digraph.nodes:
        depth_first_exploration(acyclic_digraph, node, visited, 
                                reverse_postorder)
    return reverse_postorder

In [ ]:
top = topological_sort(dag)
print(top)
nx.draw(dag, pos_dag, with_labels=True, 
        node_size=1000, node_color='lightblue')

## Connected Components of Undirected Graphs

Two vertices $u$ and $v$ are in the same connected components, if there is a path between $u$ and $v$.

In [ ]:
class ConnectedComponents:
    def __init__(self, graph):
        self.id = [None] * graph.number_of_nodes()
        self.curr_id = 0
        for node in graph.nodes:
            if self.id[node] is None:
                self.dfs(graph, node)
                self.curr_id += 1

    def dfs(self, graph, node):
        if self.id[node] is not None:
            return
        self.id[node] = self.curr_id
        for n in graph.neighbors(node):
            self.dfs(graph, n)
            
    def connected(self, node1, node2):
        return ...

    def count(self):
        return ...


<span style="color:red">*Exercise*</span>: Complete the implementation of `connected` and `count`.

In [ ]:
cc_undirected = nx.Graph()
cc_undirected.add_nodes_from([0,1,2,3,4,5,6,7,8,9])
edges_cc_undirected = [(0,1), (0,2), (1,2), (1,3), (2,3), 
                       (2,4), (3,5), (6,7), (6,8), (7,8)]
cc_undirected.add_edges_from(edges_cc_undirected)

#pos = nx.planar_layout(cc_undirected)
pos_planar = {
    0: np.array([-0.2, 0.2]),
    1: np.array([-0.2, 0.4]),
    2: np.array([0.18, 0]),
    3: np.array([-0.5, 0]),
    4: np.array([0.2, 0.15]),
    5: np.array([-0.7, 0.1]),
    6: np.array([0.5, 0.1]),
    7: np.array([0.3, -0.2]),
    8: np.array([0.9, -0.3]),
    9: np.array([0.9, 0])
}

nx.draw(cc_undirected, pos_planar, with_labels=True, node_size=1000,
        node_color='lightblue')

cc = ConnectedComponents(cc_undirected)
print("The graph has", cc.count(), "connected components.")
print("Are vertices 0 and 4 connected?", cc.connected(0,4))
print("Are vertices 3 and 7 connected?", cc.connected(3,7))

## Strongly Connected Components of Directed Graphs

We first create an example graph:

In [ ]:
scc_example = nx.DiGraph()
scc_example.add_nodes_from([0,1,2,3,4,5,6,7,8,9])
edges_scc = [(0,1), (1,2), (1,3), (2,0), (2,3), (2,4), (3,5),
             (4,2), (5,3), (6,7), (7,8), (8,6), (8,7), (8,9)]
scc_example.add_edges_from(edges_scc)

nx.draw(scc_example, pos_planar, with_labels=True, node_size=1000,
        node_color='lightblue')

We compute the strongly connected components with Kosaraju's algorithm:

In [ ]:
def compute_strongly_connected_components(directed_graph):
    reversed_graph = directed_graph.reverse(False)
    
    # compute complete reverse postorder for reversed graph
    visited = set()
    reverse_postorder = deque()
    for node in reversed_graph.nodes:
        depth_first_exploration(reversed_graph, node, visited,
                                reverse_postorder)
        
    print(reverse_postorder)
    sccs = [] # should contain one entry for each strongly connected component

    # TODO determine the strongly connected components
    # Hint: you can collect the vertices that are newly
    # visited in an exploration from the corresponding reverse
    # postorder.
    
        
        
    
    return sccs

<span style="color:red">*Exercise*</span>: Complete the implementation of `compute_strongly_connected_components`.

For testing, we color the test graph with the computed strongly connected components:

In [ ]:
sccs = compute_strongly_connected_components(scc_example)
print(sccs)

node_to_component_no = dict()
for no, component in enumerate(sccs):
    for node in component:
        node_to_component_no[node] = no
        
print(node_to_component_no)
        
nodelist = list(scc_example.nodes)
colors = [node_to_component_no[n] for n in nodelist]

nx.draw(scc_example, pos_planar, nodelist=nodelist, node_size=1000, node_color=colors)
# if drawing throws an error, try the following line that works with older versions of networkx
# nx.draw(scc_example, pos_planar, node_list=nodelist, node_size=1000, node_color=colors)
